In [3]:
#api key - 3146f5130697e385842246b291a2b25
#Host: api.meetup.com
#GET /2/groups

#! pip install requests

from __future__ import unicode_literals
import requests
import json
import time
import codecs
import sys

UTF8Writer = codecs.getwriter('utf8')
sys.stdout = UTF8Writer(sys.stdout)

def main():
       cities =[("Minneapolis", "MN")]
       api_key= "3146f5130697e385842246b291a2b25"
       for (city, state) in cities:
           per_page = 200
           results_we_got = per_page
           offset = 0
           while (results_we_got == per_page):
               # Meetup.com documentation here: http://www.meetup.com/meetup_api/docs/2/groups/
               response=get_results({"sign":"true","country":"US", "city":city, "state":state, "radius": 10, "key":api_key, "page":per_page, "offset":offset })
               time.sleep(30)
               offset += 1
               results_we_got = response['meta']['count']
               for group in response['results']:
                   category = ""
                   if "category" in group:
                       category = group['category']['name']
                   print "," .join(map(unicode, [city, group['name'].replace(","," "), group['created'], group['city'],group.get('state',""),category,group['members'], group.get('who',"").replace(","," ")]))

           time.sleep(30)

def get_results(params):
   request = requests.get("http://api.meetup.com/2/groups",params=params)
   data = request.json()
   return data

if __name__=="__main__":
       main()

## Run this script and send it into a csv:
## python meetup-pages-names-dates.py > meetup_groups.csv

Minneapolis,Pugs In The City- Twin Cities,1035836415000,Minneapolis,MN,pets/animals,784,Pugs In The City- Puglovers
Minneapolis,Minnesota Web Design+Technology Professionals,1037928288000,Minneapolis,MN,tech,1581,Web & Mobile Tech Experts
Minneapolis,Buddhism meetup,1041548688000,Minneapolis,MN,religion/beliefs,557,people
Minneapolis,Minneapolis Area Moms,1052850938000,Minneapolis,MN,parents/family,126,Mpls Area Mamas
Minneapolis,* Twin Cities Boxer Club *,1057006551000,Saint Paul,MN,pets/animals,460,Boxer Lovers
Minneapolis,Twin Cities Roleplaying Association,1064788754000,Minneapolis,MN,games,1350,Roleplayers
Minneapolis,Investing or Trading - It's Your Money,1066858789000,Minneapolis,MN,career/business,399,Investors
Minneapolis,The St. Paul Area Moms Group,1066858789000,Saint Paul,MN,parents/family,116,St. Paul Moms
Minneapolis,The Twin Cities German Language Meetup Group,1066858789000,Minneapolis,MN,language/ethnic identity,405,German Speakers
Minneapolis,The Twin Cities French Bul

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 26: ordinal not in range(128)

In [7]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working threads and a batch interval of 2 seconds
#sc = SparkContext("local[4]", "Meetup") #set to 2 or 4 processors?
ssc = StreamingContext(sc, 10) #2 = 2 seconds

# Create a DStream
lines = ssc.socketTextStream("http://api.meetup.com/2/groups", 9999)
words = lines.flatMap(lambda line: line.split(" "))
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)
wordCounts.pprint()

ssc.start()
#ssc.awaitTermination(timeout = 1)
#ssc.awaitTerminationOrTimeout(timeout = 1)
#ssc.stop(stopSparkContext = False)

False

-------------------------------------------
Time: 2016-11-25 22:09:40
-------------------------------------------

-------------------------------------------
Time: 2016-11-25 22:09:50
-------------------------------------------

-------------------------------------------
Time: 2016-11-25 22:10:00
-------------------------------------------

-------------------------------------------
Time: 2016-11-25 22:10:10
-------------------------------------------

-------------------------------------------
Time: 2016-11-25 22:10:20
-------------------------------------------

-------------------------------------------
Time: 2016-11-25 22:10:30
-------------------------------------------

-------------------------------------------
Time: 2016-11-25 22:10:40
-------------------------------------------



In [8]:
ssc.stop(stopSparkContext = False)

In [11]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

#create spark/streaming context
#del(sc)
#sc = SparkContext("local[4]", "Meetup") #set to 2 or 4 processors?
ssc = StreamingContext(sc, 5) #2 = 2 seconds

#create dstream
#pyspark.streaming.DStream(jdstream, ssc, jrdd_deserializer)

#socketTextStream(hostname, port, storageLevel=StorageLevel(True, True, False, False, 2))¶
dstr = ssc.socketTextStream("http://api.meetup.com/2/groups", 9999) #website based
dstr.count()
dstr.pprint(num=10)
#dstr = ssc.textFileStream("/usr/hdp/2.4.0.0-169/spark") #directory/file based - files need to be same type
#dstr = sc.queueStream(queueOfRDDs) #for testing - need bunch of RDDs

#dstr.cache()
#dstr.checkpoint(interval=20)

#transform
#transform(dstr, transformFunc)

#ssc.start()
#ssc.awaitTermination(timeout = 1)
#ssc.awaitTerminationOrTimeout(timeout = 1)
ssc.stop(stopSparkContext = False)

-------------------------------------------
Time: 2016-11-25 22:19:55
-------------------------------------------



In [12]:
ssc.stop(stopSparkContext = False)

-------------------------------------------
Time: 2016-11-25 22:20:00
-------------------------------------------

-------------------------------------------
Time: 2016-11-25 22:20:05
-------------------------------------------



In [ ]:
#CUSTOM RECEIVER - SCALA

package org.apache.spark.examples.streaming

import java.io.{BufferedReader, InputStreamReader}
import java.net.Socket
import java.nio.charset.StandardCharsets

import org.apache.spark.SparkConf
import org.apache.spark.internal.Logging
import org.apache.spark.storage.StorageLevel
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.streaming.receiver.Receiver

/*
 * Custom Receiver that receives data over a socket. Received bytes are interpreted as 
 * text and \n delimited lines are considered as records. They are then counted and printed.
 */
object CustomReceiver {
  def main(args: Array[String]) {
    if (args.length < 2) {
      System.err.println("Usage: CustomReceiver <hostname> <port>")
      System.exit(1)
    }

    StreamingExamples.setStreamingLogLevels()

    // Create the context with a 1 second batch size
    val sparkConf = new SparkConf().setAppName("CustomReceiver")
    val ssc = new StreamingContext(sparkConf, Seconds(1))

    // Create an input stream with the custom receiver on target ip:port and count the
    // words in input stream of \n delimited text (eg. generated by 'nc')
    val lines = ssc.receiverStream(new CustomReceiver(args(0), args(1).toInt))
    val words = lines.flatMap(_.split(" "))
    val wordCounts = words.map(x => (x, 1)).reduceByKey(_ + _)
    wordCounts.print()
    ssc.start()
    ssc.awaitTermination()
  }
}

class CustomReceiver(host: String, port: Int)
  extends Receiver[String](StorageLevel.MEMORY_AND_DISK_2) with Logging {

  def onStart() {
    // Start the thread that receives data over a connection
    new Thread("Socket Receiver") {
      override def run() { receive() }
    }.start()
  }

  def onStop() {
   // There is nothing much to do as the thread calling receive()
   // is designed to stop by itself isStopped() returns false
  }

  /** Create a socket connection and receive data until receiver is stopped */
  private def receive() {
   var socket: Socket = null
   var userInput: String = null
   try {
     logInfo("Connecting to " + host + ":" + port)
     socket = new Socket(host, port)
     logInfo("Connected to " + host + ":" + port)
     val reader = new BufferedReader(
       new InputStreamReader(socket.getInputStream(), StandardCharsets.UTF_8))
     userInput = reader.readLine()
     while(!isStopped && userInput != null) {
       store(userInput)
       userInput = reader.readLine()
     }
     reader.close()
     socket.close()
     logInfo("Stopped receiving")
     restart("Trying to connect again")
   } catch {
     case e: java.net.ConnectException =>
       restart("Error connecting to " + host + ":" + port, e)
     case t: Throwable =>
       restart("Error receiving data", t)
   }
  }
}

In [ ]:
#DATAFRAMES/SQL

"""
 Use DataFrames and SQL to count words in UTF8 encoded, '\n' delimited text received from the
 network every second.
 Usage: sql_network_wordcount.py <hostname> <port>
   <hostname> and <port> describe the TCP server that Spark Streaming would connect to receive data.
 To run this on your local machine, you need to first run a Netcat server
    `$ nc -lk 9999`
 and then run the example
    `$ bin/spark-submit examples/src/main/python/streaming/sql_network_wordcount.py localhost 9999`
"""
from __future__ import print_function

import sys

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession


def getSparkSessionInstance(sparkConf):
    if ('sparkSessionSingletonInstance' not in globals()):
        globals()['sparkSessionSingletonInstance'] = SparkSession\
            .builder\
            .config(conf=sparkConf)\
            .getOrCreate()
    return globals()['sparkSessionSingletonInstance']


if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: sql_network_wordcount.py <hostname> <port> ", file=sys.stderr)
        exit(-1)
    host, port = sys.argv[1:]
    sc = SparkContext(appName="PythonSqlNetworkWordCount")
    ssc = StreamingContext(sc, 1)

    # Create a socket stream on target ip:port and count the
    # words in input stream of \n delimited text (eg. generated by 'nc')
    lines = ssc.socketTextStream(host, int(port))
    words = lines.flatMap(lambda line: line.split(" "))

    # Convert RDDs of the words DStream to DataFrame and run SQL query
    def process(time, rdd):
        print("========= %s =========" % str(time))

        try:
            # Get the singleton instance of SparkSession
            spark = getSparkSessionInstance(rdd.context.getConf())

            # Convert RDD[String] to RDD[Row] to DataFrame
            rowRdd = rdd.map(lambda w: Row(word=w))
            wordsDataFrame = spark.createDataFrame(rowRdd)

            # Creates a temporary view using the DataFrame.
            wordsDataFrame.createOrReplaceTempView("words")

            # Do word count on table using SQL and print it
            wordCountsDataFrame = \
                spark.sql("select word, count(*) as total from words group by word")
            wordCountsDataFrame.show()
        except:
            pass

    words.foreachRDD(process)
    #ssc.start()
    #ssc.awaitTermination()